## Напишем через рандом нумпая и классы
Используем список, генерируемый в начале работы через numpy rd.randint(0, 33554432, 262144)

In [26]:
from numpy import random as rd
import numpy as np
import matplotlib.pyplot as plt


deck_names = {"green":0, "blue":1, "white":2, "yellow":3, "red":4, "black":5, "pink":6, "orange":7, "locomotive":8, "gray": 9}
deck_names_inv = ["green", "blue", "white", "yellow", "red", "black", "pink", "orange", "locomotive", "gray"]

class Card():
    def __init__(self, vagon_type):
        if vagon_type in deck_names_inv:
            self.strtype = vagon_type
            self.inttype = deck_names[vagon_type]
    
    def __str__(self):
        return self.strtype
    
    def __int__(self):
        return self.inttype
    

def deck_generate(size = 2**20, max_val = 2**27) -> list:
    return rd.randint(0, max_val, (size,110))




def get_card_from_deck(deck):
    r
    for i in range(len(deck)):
        num -= deck[i]
        if num < 0: 
            deck[i] -=1
            target[i] +=1
            return

def refresh_table(table, deck):
    while(np.sum(table) != 5):
        get


class Player():
    
    def __init__(self, need_type = "gray", need_cnt = 1):
        self.holded = np.zeros(9)
        self.count = 0
        self.ntype = need_type
        self.ncnt = need_cnt
    
    
    def __getcardfromtable
    
    def GetCard(self, nedd_type):
        
    
    
    

deck_generate()

array([[105,  38,   0,  22,   6,  48,  45,  94, 103,  30,  93,  61,  68,
        102,  95,  94,  80,  60,  15,  10, 106, 107,  43,  84,  66,  45,
         28,  76,  66,  48,  51,   6,  81,  10,  34,  86,  67,  19,  56,
         66,  39,  77,  51,  20,  28,  98,  17,   3,  46,  78,   8, 107,
         77,   4,  98,  88,  95,  19,  79,  26,  46,   5,  43,  48,  44,
          1,   2,   0,  10,  11,  65,  62, 100, 103,  52,  11,  87,  92,
         51, 107, 101,  79,  50,  68, 103,  56,  18,  57,  23, 105,  50,
         84, 102,  22,  59,   8,  17,  15,  51,  44,  13,  36,  92, 104,
         46,  65,   6, 106,  50,  50],
       [ 82,  52,  35,  75,  53,  39,  63, 107,   5,  81,  53,  57,  76,
         75, 102,  99,   0,  38,  38,  98,  79,   2,  29,  37,  45,   3,
         74,  54,  49,  65,  20,  87,   4,  60,  52,   1,   5,  45,  16,
          8, 108,  85,  56,  88,  65,  53,  95,  46,  38,  79,  91, 107,
         52,  49, 103, 104,  86,  33,  12,  68,  67,  51,  40,  49, 105,
         91,

In [28]:
Misha = Player()
Misha.holded

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
%%time

#Parametrs
need_type = "blue"
need_cnt = 4
Num_of_launches = 2 * 10**4
CPT = 2
ProgressBar = True
Get_Locomotives = "weak"

#Code
NOL = Num_of_launches
PB = ProgressBar
if NOL<100: PB = False
GL = Get_Locomotives

# fig, m_axs = plt.subplots(1, 2, figsize=(8.4*2, 1*1.8*2), constrained_layout=False)

def Turns_for_road_table_np(need_type, need_cnt, ProgressBar = PB, Num_of_launches = NOL,\
                         cards_per_turn = CPT, Get_Locomotives = GL) -> float:
    deck = nrd.randint(0, 33554432, 262144)
    turns = np.empty((Num_of_launches))

    for i in range(Num_of_launches):
        if ProgressBar: 
            if (i+1)%(Num_of_launches//10) == 0: print((i+1)//(Num_of_launches//100), end = " ")
        ct = count_turns_table_np(need_type, need_cnt, cards_per_turn, Get_Locomotives)
        turns[i] = ct
    
    if ProgressBar: print("\n")
        
#     fig, m_axs = plt.subplots(1, 1, figsize=(18, 7))   
#     m_axs.plot(np.linspace(0,Num_of_launches-1,Num_of_launches),turns)
#     plt.show()
    
    return np.average(turns)

print(Turns_for_road_table(need_type, need_cnt, Num_of_launches=NOL))
